This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# Reading NetCDF

This reader utilizes [netCDF4-python](https://unidata.github.io/netcdf4-python/).

In this example, we read NetCDF files from different sources using a generic reader from $\omega radlib's$ io module.

In [1]:
import wradlib as wrl
from wradlib.io import read_generic_netcdf
from wradlib.util import get_wradlib_data_file
import os

0.3.0


## NetCDF Format

The NetCDF format also claims to be self-describing. However, as for all such formats, the developers of netCDF also admit that "[...] the mere use of netCDF is not sufficient to make data self-describing and meaningful to both humans and machines [...]" (see [here](https://www.unidata.ucar.edu/software/netcdf/conventions.html)). Different radar operators or data distributors will use different naming conventions and data hierarchies (i.e. "data models") that the reading program might need to know about.

$\omega radlib$ provides two solutions to address this challenge. The first one ignores the concept of data models and just pulls all data and metadata from a NetCDF file ([wradlib.io.read_generic_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_generic_netcdf.html). The second is designed for a specific data model used by the EDGE software ([wradlib.io.read_edge_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_edge_netcdf.html)).

<div class="alert alert-warning">

**Note** <br>
    
For radar data in CfRadial1 or CfRadial2 format the [openradar community](https://openradarscience.org/) published [xradar](https://docs.openradarscience.org/projects/xradar/en/latest/) where xarray-based readers/writers are implemented. That particular code was ported from $\omega radlib$ to xradar. Please refer to xradar for enhancements for polar radar.    
    
From $\omega radlib$ version 1.19 `CfRadial` reading code is imported from [xradar](https://github.com/openradar/xradar)-package whenever and wherever necessary.
    
Please read the more indepth notebooks [cfradial1_backend](../backends/cfradial1_backend.ipynb) and [cfradial2_backend](../backends/cfradial2_backend.ipynb).
    
</div>

## Generic NetCDF reader (includes CfRadial)

$\omega radlib$ provides a function that will virtually read any NetCDF file irrespective of the data model: [wradlib.io.read_generic_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_generic_netcdf.html). It is built upon Python's [netcdf4](https://unidata.github.io/netcdf4-python/) library. [wradlib.io.read_generic_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_generic_netcdf.html) will return only one object, a dictionary, that contains all the contents of the NetCDF file corresponding to the original file structure. This includes all the metadata, as well as the so called "dimensions" (describing the dimensions of the actual data arrays) and the "variables" which will contains the actual data. Users can use this dictionary at will in order to query data and metadata; however, they should make sure to consider the documentation of the corresponding data model. [wradlib.io.read_generic_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_generic_netcdf.html) has been shown to work with a lot of different data models, most notably **CfRadial** (see [here](https://ncar.github.io/CfRadial/) for details). A typical call to [wradlib.io.read_generic_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_generic_netcdf.html) would look like:

In [2]:
fpath = "netcdf/example_cfradial_ppi.nc"
f = wrl.util.get_wradlib_data_file(fpath)
outdict = wrl.io.read_generic_netcdf(f)
for key in outdict.keys():
    print(key)

comment
title
Conventions
source
version
references
instrument_name
institution
field_names
history
dimensions
variables


## EDGE NetCDF

EDGE is a commercial software for radar control and data analysis provided by the Enterprise Electronics Corporation. It allows for netCDF data export. The resulting files can be read by [wradlib.io.read_generic_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_generic_netcdf.html), but $\omega radlib$ also provides a specific function,  [wradlib.io.read_edge_netcdf()](https://docs.wradlib.org/en/latest/generated/wradlib.io.netcdf.read_edge_netcdf.html) to return metadata and data as seperate objects:

In [3]:
fpath = "netcdf/edge_netcdf.nc"
f = wrl.util.get_wradlib_data_file(fpath)
data, metadata = wrl.io.read_edge_netcdf(f)
print(data.shape)
print(metadata.keys())

(360, 240)
dict_keys(['TypeName', 'DataType', 'Latitude', 'Longitude', 'Height', 'FractionalTime', 'attributes', 'NyquistVelocity-unit', 'NyquistVelocity-value', 'vcp-unit', 'vcp-value', 'radarName-unit', 'radarName-value', 'ColorMap-unit', 'ColorMap-value', 'Elevation', 'ElevationUnits', 'MissingData', 'RangeFolded', 'RadarParameters', 'PRF-unit', 'PRF-value', 'PulseWidth-unit', 'PulseWidth-value', 'MaximumRange-unit', 'MaximumRange-value', 'ConversionPlugin', 'az', 'r', 'site', 'time', 'max_range'])


In [4]:
# A little helper function for repeated tasks
def read_and_overview(filename):
    """Read NetCDF using read_generic_netcdf and print upper level dictionary keys"""
    test = read_generic_netcdf(filename)
    print("\nPrint keys for file %s" % os.path.basename(filename))
    for key in test.keys():
        print("\t%s" % key)

## CfRadial example from S-Pol research radar TIMREX campaign

See also: https://www.eol.ucar.edu/field_projects/timrex

In [5]:
filename = "netcdf/cfrad.20080604_002217_000_SPOL_v36_SUR.nc"
filename = get_wradlib_data_file(filename)
read_and_overview(filename)


Print keys for file cfrad.20080604_002217_000_SPOL_v36_SUR.nc
	Conventions
	version
	title
	institution
	references
	source
	history
	comment
	instrument_name
	site_name
	scan_name
	scan_id
	platform_is_mobile
	n_gates_vary
	dimensions
	variables


## Example PPI from Py-ART repository

See also: https://github.com/ARM-DOE/pyart/

In [6]:
filename = "netcdf/example_cfradial_ppi.nc"
filename = get_wradlib_data_file(filename)
read_and_overview(filename)


Print keys for file example_cfradial_ppi.nc
	comment
	title
	Conventions
	source
	version
	references
	instrument_name
	institution
	field_names
	history
	dimensions
	variables


## Example RHI from Py-ART repository
See also: https://github.com/ARM-DOE/pyart/

In [7]:
filename = "netcdf/example_cfradial_rhi.nc"
filename = get_wradlib_data_file(filename)
read_and_overview(filename)


Print keys for file example_cfradial_rhi.nc
	comment
	title
	Conventions
	source
	version
	references
	instrument_name
	institution
	field_names
	history
	dimensions
	variables


## Example EDGE NetCDF export format

In [8]:
filename = "netcdf/edge_netcdf.nc"
filename = get_wradlib_data_file(filename)
read_and_overview(filename)


Print keys for file edge_netcdf.nc
	TypeName
	DataType
	Latitude
	Longitude
	Height
	Time
	FractionalTime
	attributes
	NyquistVelocity-unit
	NyquistVelocity-value
	vcp-unit
	vcp-value
	radarName-unit
	radarName-value
	ColorMap-unit
	ColorMap-value
	Elevation
	ElevationUnits
	MissingData
	RangeFolded
	RadarParameters
	PRF-unit
	PRF-value
	PulseWidth-unit
	PulseWidth-value
	MaximumRange-unit
	MaximumRange-value
	ConversionPlugin
	dimensions
	variables
